In [5]:
import os
import sys
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from glob import glob

In [6]:
project_root = os.getcwd()
src_dir = os.path.join(project_root, "src")

# src-Verzeichnis in Pfad einfügen
if src_dir not in sys.path:
    sys.path.insert(0, src_dir)

# Ausgabeordner
output_dir = os.path.join(project_root, "output")
os.makedirs(output_dir, exist_ok=True)

In [7]:
# Custom-Module importieren
from otsu_global import otsu_threshold, binarize
from gray_hist import compute_gray_histogram
from Dice_Score import dice_score

In [8]:
# Alle img-Dateien finden
img_paths = sorted(glob(os.path.join(project_root, "data-git", "**", "img", "*.*"), recursive=True))

# Ergebnisse speichern
dice_results = []

for img_path in img_paths:
    rel_path = os.path.relpath(img_path, start=os.path.join(project_root, "data-git"))
    print(f"▶️  Verarbeite Bild: {rel_path}")

    # Bild laden
    pil_image = Image.open(img_path).convert("L")
    img_array = np.array(pil_image)

    # Histogramm berechnen
    hist, bin_edges = compute_gray_histogram(img_array)
    probabilities = hist / hist.sum()

    # Otsu anwenden
    threshold = otsu_threshold(probabilities)
    binary_bool = binarize(img_array, threshold)
    binary_uint8 = (binary_bool.astype(np.uint8) * 255)

    # ➕ Ground-Truth Pfad ermitteln
    # Beispiel: data-git/N2DH-GOWT1/img/t01.tif → man_seg01.tif
    dataset = rel_path.split(os.sep)[0]  # z. B. N2DH-GOWT1
    img_name = os.path.basename(img_path)  # z. B. t01.tif
    seg_number = ''.join(filter(str.isdigit, img_name))  # z. B. 01
    gt_name = f"man_seg{seg_number}.tif"
    gt_path = os.path.join(project_root, "data-git", dataset, "gt", gt_name)

    # Prüfen ob GT existiert
    if os.path.exists(gt_path):
        gt_image = Image.open(gt_path).convert("L")
        gt_array = np.array(gt_image) > 0  # Binärmaske
        dice = dice_score(gt_array, binary_bool)
        dice_results.append((rel_path, dice))
        print(f"  ✅ Dice Score: {dice:.4f}")
    else:
        print(f"  ⚠️  Keine GT-Datei gefunden: {gt_path}")
        dice_results.append((rel_path, None))

    # Bild speichern
    flat_filename = rel_path.replace(os.sep, "_").replace(".tif", "_binary.png")
    output_path = os.path.join(output_dir, flat_filename)
    Image.fromarray(binary_uint8).save(output_path)
    print(f"  💾 Gespeichert als: {output_path}\n")



▶️  Verarbeite Bild: N2DH-GOWT1\img\t01.tif
  ✅ Dice Score: 0.5705
  💾 Gespeichert als: c:\Users\victo\SynologyDrive\Studium\molekulare Biotechnologie\Heidelberg\Bioinformatik\Projekt\Otsu Treshhold\Otsu_Thresholding_LocalWork\topic01_team03\output\N2DH-GOWT1_img_t01_binary.png

▶️  Verarbeite Bild: N2DH-GOWT1\img\t21.tif
  ✅ Dice Score: 0.5738
  💾 Gespeichert als: c:\Users\victo\SynologyDrive\Studium\molekulare Biotechnologie\Heidelberg\Bioinformatik\Projekt\Otsu Treshhold\Otsu_Thresholding_LocalWork\topic01_team03\output\N2DH-GOWT1_img_t21_binary.png

▶️  Verarbeite Bild: N2DH-GOWT1\img\t31.tif
  ✅ Dice Score: 0.5680
  💾 Gespeichert als: c:\Users\victo\SynologyDrive\Studium\molekulare Biotechnologie\Heidelberg\Bioinformatik\Projekt\Otsu Treshhold\Otsu_Thresholding_LocalWork\topic01_team03\output\N2DH-GOWT1_img_t31_binary.png

▶️  Verarbeite Bild: N2DH-GOWT1\img\t39.tif
  ✅ Dice Score: 0.5830
  💾 Gespeichert als: c:\Users\victo\SynologyDrive\Studium\molekulare Biotechnologie\Heidelber

In [11]:
import csv

csv_path = os.path.join(output_dir, "dice_scores.csv")
with open(csv_path, "w", newline="", encoding="utf-8") as f:
    writer = csv.writer(f, delimiter=';')
    writer.writerow(["Bild", "DiceScore"])
    for path, score in dice_results:
        writer.writerow([path, score if score is not None else "NA"])

print(f"📄 Dice-Score-Tabelle gespeichert unter: {csv_path}")


📄 Dice-Score-Tabelle gespeichert unter: c:\Users\victo\SynologyDrive\Studium\molekulare Biotechnologie\Heidelberg\Bioinformatik\Projekt\Otsu Treshhold\Otsu_Thresholding_LocalWork\topic01_team03\output\dice_scores.csv
